In [1]:
import numpy as np 
from load_data import load_data
train, valid, test  = load_data('./mnist.pkl.gz')

D:\anaconda\lib\site-packages\numpy\core\__init__.py:29: UserWarning: loaded more than 1 DLL from .libs:
D:\anaconda\lib\site-packages\numpy\.libs\libopenblas.IPBC74C7KURV7CB2PKT5Z5FNR3SIBV4J.gfortran-win_amd64.dll
D:\anaconda\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


... loading data


In [2]:
raw_X_train = train[0]
raw_y_train = train[1]
raw_X_val = valid[0]
raw_y_val = valid[1]
raw_X_test = test[0]
raw_y_test = test[1]

In [3]:
n_train_samples = raw_X_train.shape[0]
n_val_samples = raw_X_val.shape[0]
n_test_samples = raw_X_test.shape[0]

X_train = raw_X_train.reshape(n_train_samples, 28, 28, 1)
X_val = raw_X_val.reshape(n_val_samples, 28, 28, 1)
X_test = raw_X_test.reshape(n_test_samples, 28, 28, 1)

In [4]:
from keras import utils
Y_train = utils.to_categorical(raw_y_train, num_classes=10)
Y_val = utils.to_categorical(raw_y_val, num_classes=10)
Y_test = utils.to_categorical(raw_y_test, num_classes=10)

D:\anaconda\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
from keras import Model
from keras import Input
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Dense
def mnist_model():
    X_input = Input(shape=(28, 28, 1), name='input')
    
    X = Conv2D(6, (5, 5), padding='same')(X_input)
    X =Activation('relu')(X)
    X= MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(X)
    
    X = Conv2D(16,(5, 5), padding="same")(X)
    X = Activation("relu")(X)
    X = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(X)
    
    X = Flatten()(X)
    X = Dense(120)(X)
    X = Activation("relu")(X)
    
    X = Dense(84)(X)
    X = Activation("relu")(X)
    
    X = Dense(10)(X)
    X_output = Activation("softmax")(X)
    
    model = Model(inputs = X_input, outputs = X_output)
    
    return model

In [6]:
RMSprop_model = mnist_model()

Instructions for updating:
Colocations handled automatically by placer.


In [7]:
from keras.optimizers import RMSprop
RMSprop_model.compile(loss="categorical_crossentropy", optimizer=RMSprop(lr=0.001), metrics=["accuracy"])

In [8]:
from keras.callbacks import CSVLogger
csv_logger = CSVLogger('RMSprop_training.log')

In [9]:
history = RMSprop_model.fit(X_train, Y_train, validation_data=[X_val, Y_val], batch_size=128, epochs=20, verbose=1, callbacks=[csv_logger])

Instructions for updating:
Use tf.cast instead.
Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 76s 2ms/step - loss: 0.3210 - acc: 0.9019 - val_loss: 0.1247 - val_acc: 0.9614
Epoch 2/20
50000/50000 [==============================] - 76s 2ms/step - loss: 0.0744 - acc: 0.9767 - val_loss: 0.0580 - val_acc: 0.9828
Epoch 3/20
50000/50000 [==============================] - 87s 2ms/step - loss: 0.0480 - acc: 0.9848 - val_loss: 0.0453 - val_acc: 0.9866
Epoch 4/20
50000/50000 [==============================] - 70s 1ms/step - loss: 0.0363 - acc: 0.9883 - val_loss: 0.0402 - val_acc: 0.9874
Epoch 5/20
50000/50000 [==============================] - 60s 1ms/step - loss: 0.0283 - acc: 0.9913 - val_loss: 0.0384 - val_acc: 0.9886
Epoch 6/20
50000/50000 [==============================] - 60s 1ms/step - loss: 0.0235 - acc: 0.9925 - val_loss: 0.0544 - val_acc: 0.9842
Epoch 7/20
50000/50000 [==============================] - 60s 1ms/step - loss: 0

In [10]:
(loss, accuracy) = RMSprop_model.evaluate(X_test, Y_test, batch_size=128, verbose=1)
print("accuracy: {:.2f}%".format(accuracy * 100))

10000/10000 [==============================] - 5s 544us/step
accuracy: 99.12%


In [11]:
import pandas as pd
df = pd.read_csv('RMSprop_training.log')

In [12]:
print(df)

    epoch      acc      loss  val_acc  val_loss
0       0  0.90188  0.320951   0.9614  0.124662
1       1  0.97666  0.074448   0.9828  0.057962
2       2  0.98484  0.047982   0.9866  0.045281
3       3  0.98828  0.036286   0.9874  0.040228
4       4  0.99134  0.028335   0.9886  0.038392
5       5  0.99252  0.023451   0.9842  0.054384
6       6  0.99402  0.019166   0.9904  0.040325
7       7  0.99546  0.014566   0.9886  0.043683
8       8  0.99598  0.013132   0.9880  0.044049
9       9  0.99646  0.010996   0.9897  0.045433
10     10  0.99724  0.008203   0.9889  0.052449
11     11  0.99780  0.007217   0.9889  0.056741
12     12  0.99774  0.006558   0.9873  0.066419
13     13  0.99784  0.006256   0.9902  0.050701
14     14  0.99828  0.005279   0.9904  0.051985
15     15  0.99868  0.004515   0.9882  0.078296
16     16  0.99866  0.004159   0.9887  0.057805
17     17  0.99864  0.003860   0.9898  0.058123
18     18  0.99914  0.003337   0.9910  0.055811
19     19  0.99906  0.002916   0.9905  0